In [94]:
import sys
sys.path.append('..')
import os
import re
import json
import gpt3
from datetime import datetime, timezone


#reminders = []



event_prompt_str = '''This bot automatically creates calendar appointments from sentences.

Input: Let's have a documentation sprint, 14:00 Feb 1, at Animal House
Date: 2/1/2021 2:00pm
Description: documentation sprint
Location: Animal House

Input: please set a reminder to have a welding workshop at Pirates Alley on July 28, 1:30pm
Date: 7/28/2021 1:30pm
Description: welding workshop
Location: Pirates Alley

Input: Electric unicycling repair at Mars on May 5, 10am
Date: 5/5/2021 10:00am
Description: Electric unicycling repair
Location: Mars

Input: We'll have a piano jam session at the Institute, January 15 at 3pm
Date: 1/15/2021 3:00pm
Description: piano jam session
Location: Institute

Input: {}
Date:'''


def utc_to_local(utc_dt):
    return utc_dt.replace(tzinfo=timezone.utc).astimezone(tz=None)


def save(events):
    with open('events.json', 'w') as outfile:
        json.dump(events, outfile, default=str)

def load():
    events = {}
    if os.path.exists('events.json'):
        with open('events.json') as json_file:
            events = json.load(json_file)
        for e in events:
            e['date'] = datetime.strptime(e['date'], '%Y-%m-%d %H:%M:%S')
    return events


def parse_event_string(event_str):
    response = gpt3.complete(
        event_prompt_str.format(event_str.strip()),
        stops=['\nInput:'],
        max_tokens=100,
        temperature=0.0,
        engine='curie',
        max_completions=1)
    details = re.findall('(.+)\nDescription:(.+)\nLocation:(.+)', response)
    date, description, location = [e.strip() for e in details[0]]
    return date, description, location


def add_reminder(events, title, datestr, location):
    date = datetime.strptime(datestr, '%m/%d/%Y %I:%M%p')
    events.append({
        'date': date,
        'title': title,
        'location': location
    })
    return events

    
events = load()
    
input_prompt = "Tea ceremony at Jack's house, Friday April 9 at 1:15pm"
date, description, location = parse_event_string(input_prompt)
events = add_reminder(events, description, date, location)

input_prompt = "We'll have a Magic the Gathering marathon at Tao's place, at 3:25pm Wednesday June 15, 2020"
date, description, location = parse_event_string(input_prompt)
events = add_reminder(events, description, date, location)

save(events)


In [61]:
reminders

[{'date': datetime.datetime(2020, 6, 15, 15, 25),
  'title': 'Magic the Gathering marathon',
  'location': "Tao's place"},
 {'date': datetime.datetime(2021, 4, 9, 13, 15),
  'title': 'Tea ceremony',
  'location': "Jack's house"}]

In [93]:

print(events)

2020-06-15 15:25:00
2021-04-09 13:15:00
[{'date': datetime.datetime(2020, 6, 15, 15, 25), 'title': 'Magic the Gathering marathon', 'location': "Tao's place"}, {'date': datetime.datetime(2021, 4, 9, 13, 15), 'title': 'Tea ceremony', 'location': "Jack's house"}]
